# Fator de Visão de Céu a partir do LiDAR com Ray Casting

Essa tentativa vai utililzar uma técnica análoga ao Ray Casting para tentar simplificar e melhorar a performance

In [245]:
import pdal
import numpy as np

In [246]:
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime


In [247]:
pipeline="""{
  "pipeline": [
    {
        "type": "readers.las",
        "filename": "arquivos/MDS/MDS_3314-231.laz"
    },
    {
        "type": "filters.sample",
        "radius": 1
    }
  ]
}"""

In [248]:
r = pdal.Pipeline(pipeline)
r.validate()
r.execute()
arrays = r.arrays

In [249]:
len(arrays[0])

471962

In [250]:
arrays[0][0]

(333499.29, 7395564.35, 735.34, 18, 1, 1, 0, 0, 6, -14., 17, 6, 356569.36606547)

In [251]:
r.schema

{'schema': {'dimensions': [{'name': 'X', 'size': 8, 'type': 'floating'},
   {'name': 'Y', 'size': 8, 'type': 'floating'},
   {'name': 'Z', 'size': 8, 'type': 'floating'},
   {'name': 'Intensity', 'size': 2, 'type': 'unsigned'},
   {'name': 'ReturnNumber', 'size': 1, 'type': 'unsigned'},
   {'name': 'NumberOfReturns', 'size': 1, 'type': 'unsigned'},
   {'name': 'ScanDirectionFlag', 'size': 1, 'type': 'unsigned'},
   {'name': 'EdgeOfFlightLine', 'size': 1, 'type': 'unsigned'},
   {'name': 'Classification', 'size': 1, 'type': 'unsigned'},
   {'name': 'ScanAngleRank', 'size': 4, 'type': 'floating'},
   {'name': 'UserData', 'size': 1, 'type': 'unsigned'},
   {'name': 'PointSourceId', 'size': 2, 'type': 'unsigned'},
   {'name': 'GpsTime', 'size': 8, 'type': 'floating'}]}}

## Importando dados para Pandas

In [252]:
import pandas as pd

In [253]:
df = pd.DataFrame(arrays[0])

In [254]:
df

,X,Y,Z,Intensity,ReturnNumber,NumberOfReturns,ScanDirectionFlag,EdgeOfFlightLine,Classification,ScanAngleRank,UserData,PointSourceId,GpsTime
0,333499.29,7395564.35,735.34,18,1,1,0,0,6,-14.0,17,6,356569.366065
1,333499.23,7395565.79,735.41,18,1,1,0,0,6,-14.0,17,6,356569.366090
2,333499.20,7395566.79,735.25,15,1,1,0,0,6,-14.0,17,6,356569.366105
3,333499.15,7395568.21,735.37,14,1,1,0,0,6,-14.0,17,6,356569.366129
4,333499.10,7395569.68,735.30,17,1,1,0,0,6,-15.0,17,6,356569.366153
...,...,...,...,...,...,...,...,...,...,...,...,...,...
471957,333523.36,7395570.78,735.52,2,1,2,1,0,19,13.0,40,6,363044.324524
471958,333521.03,7395570.91,731.07,5,1,1,1,0,6,13.0,40,6,363044.342732
471959,333520.96,7395571.00,732.50,8,1,1,1,0,6,13.0,40,6,363044.342740
471960,333521.09,7395571.84,731.83,18,1,1,1,0,6,13.0,40,6,363044.360909


In [179]:
df.Classification.unique()

array([ 6, 20,  2, 19,  5,  9], dtype=uint8)

## Pre-Processando o Conjunto de Dados

In [255]:
pd.DataFrame.from_records(df[(df.Classification == 2) | (df.Classification == 6)]\
                          [['X', 'Y', 'Z']]).dtypes

X    float64
Y    float64
Z    float64
dtype: object

In [256]:
df_1m = \
df[(df.Classification == 2) | (df.Classification == 6)]\
                          [['X', 'Y', 'Z']]\
.astype({'X': 'int32', \
        'Y': 'int32', \
        'Z': 'int16'})\
# .set_index(['X', 'Y'])

In [257]:
len(df_1m)

316682

In [258]:
## Deslocar X e Y de acordo com Azimute e inclinação

x, y = 100, 100

hag = 10 #Height Above Ground
z = 100
inclinacao = 45
azimute = 90

sombra = np.tan(inclinacao / 180. * np.pi) * z
print(sombra)

x = np.cos(azimute/180 * np.pi) * sombra
y = np.sin(azimute/180 * np.pi) * sombra

print(x, y)


99.99999999999999
6.123233995736765e-15 99.99999999999999


In [263]:
df_1m['X_proj'] = np.rint(df_1m.X + (np.cos(azimute/180 * np.pi) * np.tan(inclinacao / 180. * np.pi) * df_1m.Z)).astype('int32')
df_1m['Y_proj'] = np.rint(df_1m.Y + (np.sin(azimute/180 * np.pi) * np.tan(inclinacao / 180. * np.pi) * df_1m.Z)).astype('int32')

In [264]:
df_1m

,X,Y,Z,X_proj,Y_proj
0,333499,7395564,735,333499,7396299
1,333499,7395565,735,333499,7396300
2,333499,7395566,735,333499,7396301
3,333499,7395568,735,333499,7396303
4,333499,7395569,735,333499,7396304
...,...,...,...,...,...
471956,333520,7395569,730,333520,7396299
471958,333521,7395570,731,333521,7396301
471959,333520,7395571,732,333520,7396303
471960,333521,7395571,731,333521,7396302


In [300]:
XY_projZmax = df_1m[['X_proj', 'Y_proj', 'Z']].groupby(['X_proj', 'Y_proj'],  as_index=False).max()

In [316]:
df_1m[['X_proj', 'Y_proj', 'Z']].groupby(by=['X_proj', 'Y_proj']).max()

Z
X_proj Y_proj      
333058 7395745  748
       7395747  748
       7395751  757
       7395752  767
       7395753  767
...             ...
333600 7396324  758
       7396325  758
       7396326  757
       7396327  757
       7396328  757

[173434 rows x 1 columns]

In [291]:
seila = df_1m[['X_proj', 'Y_proj', 'Z']].groupby(['X_proj', 'Y_proj']).groups

In [272]:
df_1m.loc[df_1m.X == XY_projZmax.X_proj]

ValueError: Can only compare identically-labeled Series objects

In [213]:
fusos = 60
inclinacoes = 45

In [217]:
## Para cada fuso
    ## Para cada inclinacao
        ## CAlcula valores inteiros de X e Y inclinados e no Azimute
        

In [231]:
np.array([df_1m.X, df_1m.Y]).shape

(2, 316682)

array([333058, 333059, 333060, 333061, 333062, 333063, 333064, 333065,
       333066, 333067, 333068, 333069, 333070, 333071, 333072, 333073,
       333074, 333075, 333076, 333077, 333078, 333079, 333080, 333081,
       333082, 333083, 333084, 333085, 333086, 333087, 333088, 333089,
       333090, 333091, 333092, 333093, 333094, 333095, 333096, 333097,
       333098, 333099, 333100, 333101, 333102, 333103, 333104, 333105,
       333106, 333107, 333108, 333109, 333110, 333111, 333112, 333113,
       333114, 333115, 333116, 333117, 333118, 333119, 333120, 333121,
       333122, 333123, 333124, 333125, 333126, 333127, 333128, 333129,
       333130, 333131, 333132, 333133, 333134, 333135, 333136, 333137,
       333138, 333139, 333140, 333141, 333142, 333143, 333144, 333145,
       333146, 333147, 333148, 333149, 333150, 333151, 333152, 333153,
       333154, 333155, 333156, 333157, 333158, 333159, 333160, 333161,
       333162, 333163, 333164, 333165, 333166, 333167, 333168, 333169,
      

## Calculando o Azimute entre dois pontos

Suponhamos que fossemos calcular o azimute para cada ponto poderíamos fazer utilizando a seguinte função

In [12]:
import math

def Az(x1,y1,x2,y2):
    deg = math.degrees(math.atan2((x2 - x1),(y2 - y1)))
    if (deg < 0):
        deg += 360.0
    return deg

In [13]:
Az(1, 1, 1, 0)

180.0

## Calculando a Altura ou Elevação do ponto em relação ao Zenite

Da mesma forma poderíamos calcular a Altura/Elevação para cada um dos pontos usando a seguinte função

In [14]:
from scipy.spatial import distance

In [15]:
## Calculando altura
def altura_graus(x1, y1, z1, x2, y2, z2):
    altura = z2 - z1
    distancia = distance.euclidean([x1, y1], [x2, y2])
    return math.degrees(math.atan2(altura, distancia))

altura_graus(0,0,0,1,1,1)

35.264389682754654

## Gerando pontos 1000 pontos x,y,z 

Apesar de termos diversas dimensões disponíveis para os calculos vamos nos ater em X, Y e Z e na Classificação para poder agrupar o que é Ground e o Que é edificação

In [17]:
df_pts = pd.DataFrame.from_records(df[(df.Classification == 2) | (df.Classification == 6)][['X', 'Y', 'Z']].head(1000))

In [18]:
df_pts

,X,Y,Z
0,333499.29,7395564.35,735.34
1,333499.23,7395565.79,735.41
2,333499.20,7395566.79,735.25
3,333499.15,7395568.21,735.37
4,333499.10,7395569.68,735.30
...,...,...,...
995,333478.98,7395552.47,724.99
996,333478.48,7395558.73,732.93
997,333478.44,7395558.95,733.96
998,333478.21,7395558.71,740.46


## Calculando o Azimute com Matriz de pontos

Portanto agora que temos os pontos de origem e os pontos alvo podemos calcular o Azimute. Porém se fizermos isso ponto a ponto não vamos usar os recursos computacionais de vetorização que promovem grande eficiência computacional.
Ainda não será exatamente essa função que vamos utilzar em produção, mas para fins didáticos e de aprendizado vamos mante-las.

In [19]:
import numpy as np

def np_az(nd_building, nd_ground):
    delta_x = nd_building[:, 0] - nd_ground[0, 0]
    delta_y = nd_building[:, 1] - nd_ground[0, 1]
    degree = np.arctan2(delta_x, delta_y) * 180 / np.pi
    degree[np.less(degree, 0)] += 360.0
    return degree

In [20]:
## Gerando dados de teste
test_ground = np.array([[0.,0.,0.], [1.,1.,1.]])
test_building = np.array([[0.,2.,2.], [2.,0,2.], [0.,-2.,2.], [-2.,0,18787.], [-2.,9899.,0.1]])

In [21]:
az = np_az(test_building, test_ground)
az

array([  0.        ,  90.        , 180.        , 270.        ,
       359.98842393])

## Calculando a altura/elevação com matriz de pontos

Da mesma forma vamos fazer com a altura/elevação

In [22]:
## Calculando altura/elevação

from scipy.spatial import distance

def np_altura(nd_building, nd_ground):
    altura = nd_building[:, 2] - nd_ground[0, 2]
    distancia = np.sqrt(np.sum((nd_building[:, [0,1]] - nd_ground[0, [0,1]])**2, axis=1))
    return np.arctan2((nd_building[:, 2] - nd_ground[0, 2]), distancia) * 180 / np.pi

In [24]:
altura = np_altura(test_building, test_ground)
altura

array([4.50000000e+01, 4.50000000e+01, 4.50000000e+01, 8.99939005e+01,
       5.78803701e-04])

In [25]:
nd_coordenadas = np.array([az, altura])
nd_coordenadas.transpose().shape

(5, 2)

## Utilizando os dados de Building

As funcões de calculo de Azimute e Altura/Elevação produzidas até agora aqui nesse NoteBook precisam ser iteradas sobre cada linha de ponto de ground. Portanto precisamos refatorar essas funções para trabalhar e iterar sobre cada um dos pontos utilizando o potencial computacional do NumPy, ou seja, trabalhando com vetorização.

In [26]:
nd_building = df_building.to_numpy()
nd_ground = df_ground.to_numpy()

In [27]:
len(nd_building)

300792

In [28]:
#def calc_svf(nd_building, nd_ground):
#    az = np_az(nd_building, nd_ground)
#    altura = np_altura(nd_building, nd_ground)
#    coordenadas = np.array([az, altura])
#    return coordenadas.transpose()

In [29]:
#calc_svf(nd_building, nd_ground)

### Refatorando as funções de Azimute e Altura para trabalhar com o vetorização de Ground e Building

Para fins computacionais e menos iterações de laço que tendem a não ser tão eficientes, vamos tentar usar os recursos de vetorização e evitar o laço repetitivo

In [30]:
# def np_coordenadas(nd_building, nd_ground):
#     n2d_ground_z = nd_ground[:, 2].reshape(-1, 1)
#     n2dt_building_z = nd_building[:, 2].reshape(-1, 1).transpose()
    
#     nd_altura = np.subtract(n2dt_building_z, n2d_ground_z)
    
#     n2d_ground_x = nd_ground[:, 0].reshape(-1, 1)
#     n2dt_building_x = nd_building[:, 0].reshape(-1, 1).transpose()
#     nd_delta_x = np.subtract(n2dt_building_x, n2d_ground_x)
    
#     n2d_ground_y = nd_ground[:, 1].reshape(-1, 1)
#     n2dt_building_y = nd_building[:, 1].reshape(-1, 1).transpose()
#     nd_delta_y = np.subtract(n2dt_building_y, n2d_ground_y)
    
#     nd_distancia = np.hypot(nd_delta_x, nd_delta_y)
    
#     nd_elevacao = np.degrees(np.arctan2(nd_altura, nd_distancia))
#     #nd_elevacao[np.less(nd_elevacao, 0.)] += 360.0
    
#     nd_azimute = np.arctan2(nd_delta_x, nd_delta_y) * 180 / np.pi
#     nd_azimute[np.less(nd_azimute, 0.)] += 360.0
    
#     return nd_azimute, nd_elevacao

In [31]:
## Gerando dados de teste
test_ground = np.array([[0.,0.,0.], [-1.,-1.,-1.], [1.,1.,1.]], dtype='float32')
test_building = np.array([[0.,2.,2.], [2.,0,2.], [0.,-2.,2.], [-2.,0,18787.], [-2.,9899.,0.1]], dtype='float32')

In [32]:
# azimute, elevacao = \
# np_coordenadas(nd_building, nd_ground)
# print("Distância: ")
# print(elevacao.shape)
# print(elevacao.size)
# print("Elevacao: ")
# print(azimute.shape)
# print(azimute.size)

In [33]:
# nd_coordenadas = np.stack((azimute, elevacao), axis=1)

In [34]:
# nd_coordenadas

### Menos linhas, mais legibilidade, talvez mais eficiência

Podemos ainda refatorar o código acima e reduzir algumas linhas de código, trabalhando desde o começo com 3 dimensões. Isso ainda pode acarretar alguma melhora na performance computacional mas não é a intenção por enquanto.

In [36]:
def np_coordenadas(nd_building, nd_ground):
    n2d_ground = nd_ground[:, :].reshape(-1, 3).astype('float32')
    n2d_ground = n2d_ground.reshape(-1, 3, 1)

    n2dt_building = nd_building[:, :].reshape(-1, 3).astype('float32')
    n2dt_building = n2dt_building.reshape(-1, 3, 1).transpose()

    nd_delta = np.subtract(n2dt_building, n2d_ground)
    
    nd_altura = nd_delta[:,2,:]
    
    nd_distancia = np.hypot(nd_delta[:,0,:], nd_delta[:,1,:])
    
    nd_elevacao = np.degrees(np.arctan2(nd_altura, nd_distancia))
    nd_elevacao[np.less(nd_elevacao, 0.)] += 360.0
    
    nd_azimute = np.arctan2(nd_delta[:,1,:], nd_delta[:,0,:]) * 180 / np.pi
    nd_azimute[np.less(nd_azimute, 0.)] += 360.0
    
    return nd_azimute, nd_elevacao

In [45]:
n2d_ground = nd_ground[:, :].reshape(-1, 3).astype('float32')
n2d_ground = n2d_ground.reshape(-1, 3, 1)

In [38]:
n2dt_building = nd_building[:, :].reshape(-1, 3).astype('float32')
n2dt_building = n2dt_building.reshape(-1, 3, 1).transpose()

In [39]:
nd_delta = np.subtract(n2dt_building, n2d_ground)

In [47]:
nd_altura = nd_delta[:,2,:]

In [51]:
nd_distancia = np.hypot(nd_delta[:,0,:], nd_delta[:,1,:])

In [54]:
nd_elevacao = np.degrees(np.arctan2(nd_altura, nd_distancia))
nd_elevacao[np.less(nd_elevacao, 0.)] += 360.0

In [56]:
nd_elevacao.shape

(1000, 300792)

In [ ]:
azimute, elevacao = np_coordenadas(nd_building, nd_ground)
print("Distância: ")
print(elevacao.shape)
print(elevacao.size)
print("Elevacao: ")
print(azimute.shape)
print(azimute.size)

In [ ]:
nd_ground.reshape(-1, 3, 1)

In [ ]:
nd_coordenadas = np.stack((azimute, elevacao), axis=1)

In [ ]:
nd_coordenadas

## Calculando o SVF

Agora para calcular o SVF vamos dividir a semi-esfera visível do céu em N fusos, definido pela variável`quantidade_de_fusos`.
E cada um dos fusos terá uma área esférica visível definido pela função `proporcao_visivel_no fuso`.
Depois de disponível essa função vamos iterar ela sobre todos os pontos de ground resultando no retorno da função `proporcao_visivel_total`.

In [ ]:
quantidade_de_fusos = 36